<a href="https://colab.research.google.com/github/yaraeslamm/Motion_Analysis/blob/main/Motion_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, we need to mount Google Drive to access the video
from google.colab import drive
import cv2
import numpy as np
import os

# Mount Google Drive
drive.mount('/content/drive')


# Define the paths
base_path = "/content/drive/My Drive"
video_path = os.path.join(base_path, "wave.avi")  # Replace with your video file name in Drive
mei_folder = os.path.join(base_path, "AVP/MEI")
mhi_folder = os.path.join(base_path, "AVP/MHI")

# Create folders for saving MEI and MHI frames
os.makedirs(mei_folder, exist_ok=True)
os.makedirs(mhi_folder, exist_ok=True)

# Initialize MEI and MHI computation
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    print("Error: Unable to read video.")
    cap.release()
    exit()

# Step 2: Define dimensions and initialize variables
height, width = frame.shape[:2]
mei = np.zeros((height, width), dtype=np.uint8)  # Motion Energy Image (binary)
mhi = np.zeros((height, width), dtype=np.float32)  # Motion History Image
timestamp = 1  # Initialize timestamp for MHI

# Parameters
duration = 30  # Decay duration for MHI in frames
threshold = 25  # Threshold for detecting motion

frame_idx = 0  # Frame index for saving images

# Step 3: Process the video frame by frame
while ret:
    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to detect motion (binarize)
    if frame_idx == 0:
        prev_frame = gray_frame  # Initialize with the first frame
    else:
        # Compute frame difference
        diff_frame = cv2.absdiff(prev_frame, gray_frame)
        _, binary_frame = cv2.threshold(diff_frame, threshold, 255, cv2.THRESH_BINARY)

        # Update MEI
        mei = cv2.bitwise_or(mei, binary_frame)

        # Update MHI
        mhi[binary_frame > 0] = timestamp  # Update timestamp for moving pixels
        mhi[mhi > 0] -= 1  # Decrease timestamp for older motion

        # Normalize MHI for visualization (scale to [0, 255])
        mhi_normalized = np.uint8((mhi / duration) * 255)

        # Save intermediate MEI and MHI frames
        cv2.imwrite(os.path.join(mei_folder, f"mei_frame_{frame_idx:04d}.jpg"), mei)
        cv2.imwrite(os.path.join(mhi_folder, f"mhi_frame_{frame_idx:04d}.jpg"), mhi_normalized)

    # Prepare for next frame
    prev_frame = gray_frame.copy()
    frame_idx += 1
    timestamp += 1
    ret, frame = cap.read()

cap.release()

# Step 4: Save final MEI and MHI templates
cv2.imwrite(os.path.join(base_path, "MEI.png"), mei)
cv2.imwrite(os.path.join(base_path, "MHI.png"), mhi_normalized)

print(f"Processing complete. Outputs saved in {base_path}")


Mounted at /content/drive
Processing complete. Outputs saved in /content/drive/My Drive
